In [16]:
import requests
import time
import os, json
import csv
import concurrent.futures
# from functools import partial
import pandas as pd

In [17]:
# url = "https://refract.fosfor.com/magiccodermodel/6188a3e2-bc43-444e-bfaa-8159fca71b3a/score"
url = "http://vllm-serving-service:8888/v1/completions"
# lltoken = "eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJ4WTdTd3k5UE1xaXRDQmNSMm5qcVl6bmoxS3NqZzV3TmdOV0xDVzdyUkhvIn0.eyJleHAiOjE3MzA4ODM3MzgsImlhdCI6MTY5OTI2MTMzOCwiYXV0aF90aW1lIjoxNjk5MjUyMDQ0LCJqdGkiOiJmN2EzMzQwYy1kNDQwLTRlMzUtYjk2ZS04YzBiMTc0Y2RhODAiLCJpc3MiOiJodHRwczovL3JlZnJhY3QtbG9naW4uZm9zZm9yLmNvbS9hdXRoL3JlYWxtcy9tb3NhaWMiLCJhdWQiOlsibW9zYWljLWdhdGVrZWVwZXIiLCJhY2NvdW50Il0sInN1YiI6IjZjMjU4MWU3LWZmMTItNDljNy04MDJmLWI2ZjQzOWQxZDIwMSIsInR5cCI6IkJlYXJlciIsImF6cCI6Im1vc2FpYy1nYXRla2VlcGVyIiwic2Vzc2lvbl9zdGF0ZSI6IjBhY2Y3YWZhLTVmMzMtNGRhZS05OGM3LTQyZDQwYTdlZTM2NiIsImFsbG93ZWQtb3JpZ2lucyI6WyIqIl0sInJlYWxtX2FjY2VzcyI6eyJyb2xlcyI6WyJNTE9QUyIsImxvbmdfbGl2ZWRfdG9rZW4iLCJzcGVjdHJhLWRldmVsb3BlciIsImRlZmF1bHQtcm9sZXMtbW9zYWljIiwicmVmcmFjdC1kZXZlbG9wZXIiLCJvZmZsaW5lX2FjY2VzcyIsImFkbWluIiwidW1hX2F1dGhvcml6YXRpb24iLCJyZWZyYWN0LWFkbWluIl19LCJyZXNvdXJjZV9hY2Nlc3MiOnsiYWNjb3VudCI6eyJyb2xlcyI6WyJtYW5hZ2UtYWNjb3VudCIsIm1hbmFnZS1hY2NvdW50LWxpbmtzIiwidmlldy1wcm9maWxlIl19fSwic2NvcGUiOiJvcGVuaWQgZW1haWwgcHJvZmlsZSIsInNpZCI6IjBhY2Y3YWZhLTVmMzMtNGRhZS05OGM3LTQyZDQwYTdlZTM2NiIsImVtYWlsX3ZlcmlmaWVkIjp0cnVlLCJuYW1lIjoiUmVmcmFjdCBCRlNJIiwicHJlZmVycmVkX3VzZXJuYW1lIjoicmVmcmFjdC5iZnNpQGZvc2Zvci5jb20iLCJnaXZlbl9uYW1lIjoiUmVmcmFjdCIsImZhbWlseV9uYW1lIjoiQkZTSSIsImVtYWlsIjoicmVmcmFjdC5iZnNpQGZvc2Zvci5jb20ifQ.b6SYLgjo9Veo3GmJ8eZjCTNupQjpfMhzsoXdYjWwRtvRnNjBfx0gOqcugO9OcGn-mm8wwpSGI5uiL30-I6SdWBjsf1ur6GztoX7j-nP_3SrJJn3UhNNqIO8LbsPi5gGRTzWtnfjz92BF1YaCXxQwPY0P_aa8vJ6JxZz5Uctn9aIPIJZZnnjC_GPXtXurmshM_tEN2kwCjhEyr7wYzRqUoMtBGfpLjZREBzgZY-x6JyYiXNtycb1d6PFcCXf7nJVV8ienEC_x7OuciDzfeqd-SQnImvAHH7rqFdi9smBN08AbkDS2uAbMrokHrmbiBpaimrR013VwCWz2KL5QYlWleA"
request_payload_file = '/data/Magicoder_payloads.txt'
report_path = '/data/vLLM_5Concurrency_1gpu.csv'
df = pd.DataFrame(columns=['Response', 'Response_status', 'Time (seconds)'])

In [18]:
payload = {"prompt": "Problem: Write a function in python to check if two strings are anagrams.Input: Strings s1 and s2.Output: Boolean, indicating whether s1 and s2 are anagrams.", 
           "max_tokens": 200,
           "temperature": 0,
           "model": "/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66"}

In [19]:
def read_payloads(file_path):
    with open(file_path, 'r') as file:
        for line in file:
            yield line.strip()

In [20]:
def my_decorator(func):
    def calculate_execution_time(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        execution_time = end_time - start_time
        data = {'Response':result.text, 'Response_status':result.status, 'Time (seconds)':f'{execution_time:.6f}'}
        print(data)
        df = df.append(data, ignore_index=True)
        print(f"Execution time of {result}: {execution_time:.6f} seconds - Response===> {_.text} \n\n")
        print(df)
        return result, _
    return calculate_execution_time

In [21]:
def save_report(file_path, df):
    file_exists = os.path.exists(file_path)
    df.to_csv(file_path, index=False)

In [22]:
@my_decorator
def inference(payload_data=None):
    headers = {
        'Content-Type': 'application/json'
    }
    try:
        response = requests.request("POST", url, headers=headers, json=payload_data)
        print(f"Prompt==> {payload_data['prompt']} \n {response.text} \n\n")
        return response
    except Exception as e:
        print(e)
        

#Model Inference(Request)
if __name__ == "__main__":
    # Using the generator function to read lines  {**payload, 'prompt': {prompt}}
    prompts = read_payloads(request_payload_file)
    for i in range(1):
        with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
            futures = [executor.submit(inference, {**payload, 'prompt': prompt}) for prompt in prompts]
            concurrent.futures.wait(futures)
    save_report(report_path, df)

Prompt==> {"payload":"Problem: Write a function in python to reverse a given string.Input: String s.Output: String, the reversed version of s."} 
 {"id":"cmpl-c3c4c1d4f6bd454a84005ba1eda94eb7","object":"text_completion","created":432901,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\ndef reverse_string(s):\n    return s[::-1]\n\ns = input(\"Enter a string: \")\nprint(\"Reversed string is: \", reverse_string(s))","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":36,"total_tokens":83,"completion_tokens":47}} 


Prompt==> {"payload":"Problem: Write a function in java to calculate the factorial of a given non-negative integer.Input: Non-negative integer n. Output: Integer, the factorial of n."} 
 {"id":"cmpl-bbd98ceaba514ae49a34f6b1aae333dd","object":"text_completion","created":432903,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd

Prompt==> {"payload":"Problem: Write a function in python to transpose a given matrix.Input: 2D list representing a matrix.Output: 2D list, the transpose of the input matrix."} 
 {"id":"cmpl-c02aad26616d4605b9ead4e2ec66b94a","object":"text_completion","created":432901,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\ndef transpose(matrix):\n    return [[matrix[j][i] for j in range(len(matrix))] for i in range(len(matrix[0]))]\n\n# Test the function\nmatrix = [[1, 2, 3], [4, 5, 6], [7, 8, 9]]\nprint(transpose(matrix))","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":46,"total_tokens":132,"completion_tokens":86}} 


Prompt==> {"payload":"Problem: Write a function in c++ to generate the nth Fibonacci number.Input: Non-negative integer n.Output: Integer, the nth Fibonacci number."} 
 {"id":"cmpl-4ca33dfd0d6e46668a2f39733f10b817","object":"text_completion","created":432906,"mode

Prompt==> {"payload":"Problem: Write a function in javascript to perform binary search on a sorted list.Input: Sorted list arr and target value target.Output: Integer, the index of target in arr (or -1 if not found)."} 
 {"id":"cmpl-9799b3d03ae44ad18d6b7593bfa5b522","object":"text_completion","created":432914,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":52,"total_tokens":53,"completion_tokens":1}} 


Prompt==> {"payload":"Problem: Write a function in javascript to generate the nth Fibonacci number.Input: Non-negative integer n.Output: Integer, the nth Fibonacci number."} 
 {"id":"cmpl-f0bcbdab3d22460b8d9df0ac7c521599","object":"text_completion","created":432911,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\nfunction fibonacci(n) {

Prompt==> {"payload":"Problem: Write a function in c++ to check if two strings are anagrams.Input: Strings s1 and s2.Output: Boolean, indicating whether s1 and s2 are anagrams."} 
 {"id":"cmpl-80e5d84eb08549cab0b5f5503d3bb701","object":"text_completion","created":432906,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\n#include <bits/stdc++.h>\nusing namespace std;\n\nbool areAnagram(string s1, string s2)\n{\n    int count[256] = {0};\n    if (s1.length() != s2.length())\n        return false;\n    for (int i = 0; i < s1.length(); i++) {\n        count[s1[i]]++;\n        count[s2[i]]--;\n    }\n    for (int i = 0; i < 256; i++)\n        if (count[i] != 0)\n            return false;\n    return true;\n}\n\nint main()\n{\n    string s1 = \"listen\";\n    string s2 = \"silent\";\n    if (areAnagram(s1, s2))\n        cout","logprobs":null,"finish_reason":"length"}],"usage":{"prompt_tokens":51,"t

Prompt==> {"payload":"Problem: Implement form validation in CSS to ensure that specific fields are filled out correctly.Input: Form input values.Output: Boolean, indicating whether the form is valid."} 
 {"id":"cmpl-689a6c1801e04755b2e04ba1a37e506f","object":"text_completion","created":432920,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":42,"total_tokens":43,"completion_tokens":1}} 


Prompt==> {"payload":"Problem: Create an image slider in CSS that displays images in a sequence with next and previous buttons.Input: Array of image URLs.Output: None (update the displayed image)."} 
 {"id":"cmpl-3545ea6430b24aaaa661465622577476","object":"text_completion","created":432920,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"","logprobs":null,"

Prompt==> {"payload":"Problem: Implement an autocomplete feature in angular for a search input field.Input: String, user input in the search field.Output: None (update the autocomplete suggestions)."} 
 {"id":"cmpl-e037f28d56e24453946a2696e96a7572","object":"text_completion","created":432921,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":48,"total_tokens":49,"completion_tokens":1}} 


Prompt==> {"payload":"Problem: Write a function in angular to fetch data from an API and display it on a webpage.Input: API endpoint URL.Output: None (update the webpage with fetched data)."} 
 {"id":"cmpl-a26c4853a5ea492180fe1f43d9cf863d","object":"text_completion","created":432921,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"","logprobs":null,"finish_r

Prompt==> {"payload":"Problem: Build a function in jQuery to populate an HTML table with data from an array of objects.Input: Array of objects, each representing a row in the table.Output: None (update the HTML table)."} 
 {"id":"cmpl-fe1c837c57024c27b49c7b29b3c5c39c","object":"text_completion","created":432922,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":51,"total_tokens":52,"completion_tokens":1}} 


Prompt==> {"payload":"Problem: Implement a function in jQuery to validate an email address.Input: String, email address.Output: Boolean, indicating whether the email address is valid."} 
 {"id":"cmpl-7afde80199c742b5a2277889c4db2386","object":"text_completion","created":432922,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"","logprobs":

In [ ]:
# import aiohttp
# import asyncio
# import time

# start_time = time.time()
# headers={"Content-type":"application/json"} 
# async def main():

#     async with aiohttp.ClientSession(trust_env = True, headers=headers) as session:

# #         for payload in payloads:
# #             url='https://refract.fosfor.com/openchatfinetunedimqlgenerator/d0721c53-c9f5-4d79-a988-2d21df46198f/score' 
#         async with session.post(url, json=payload) as resp:
#                 mql = await resp.json(content_type=None)
#                 print(mql)

# await main()
# print("--- %s seconds ---" % (time.time() - start_time))